In [37]:
%load_ext autoreload
%autoreload 2

import numpy as np
from sklearn import svm
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from kernelbiome.kernels_jax import kmat_aitchison_rbf
from kernelbiome.helpers_jax import rbf_median_heruistic, wrap
from kernelbiome.nested_cv import run_nested_cv_kfold, run_experiments
from kernelbiome.utils_cv import default_kernel_params_grid, get_kmat_with_params
from kernelbiome.utils_result import make_result_table, top_models_in_each_group

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Simulate a small dataset (regression)

In [16]:
seed_num = 2022
rng = np.random.default_rng(seed=seed_num)

n = 200
p = 10
X = np.exp(rng.normal(0, 1, n*p).reshape((n, p)))
X /= X.sum(axis=1)[:, None]
b = rng.uniform(-2, 2, p)
y = np.log(X).dot(b)

#### Calculate a kernel matrix

See `kernels_jax.py` for more details.

In [17]:
g = rbf_median_heruistic(X, clr=True, eps=1e-5)
K = kmat_aitchison_rbf(X, X, g)

#### Fit a kernel SVM with a customized kernel

In [24]:
# precompute kernel matrix
g = rbf_median_heruistic(X, clr=True, eps=1e-5)
K = kmat_aitchison_rbf(X, X, g)

# specify the estimator and fit the model
estimator = svm.SVR(kernel='precomputed', C=10)
estimator.fit(K, y)

# evaluate results
print(mean_squared_error(y_true = y, y_pred=estimator.predict(K)))

0.011928680338978388


#### Hyperparameter selection with GridSearchCV

In [19]:
# precompute kernel matrix
g = rbf_median_heruistic(X, clr=True, eps=1e-5)
K = kmat_aitchison_rbf(X, X, g)

# specify the estimator & the hyperparameter grid, fit the model
estimator = svm.SVR(kernel='precomputed')
estimator_param_grid = dict(C=[10**x for x in [-3, -2, -1, 0, 1, 2, 3]])
gscv = GridSearchCV(estimator=estimator, param_grid=estimator_param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=4, verbose=0)
gscv.fit(K, y)

# evaluate results
print(gscv.score(K, y))
print(gscv.best_estimator_)

-0.011928680338978388
SVR(C=10, kernel='precomputed')


#### Report nested CV results

The outer CV can be either kfold or leave-one-group-out. For classification, kfold CV is stratified.

The function `run_nested_cv_kfold` (or `run_nested_cv_logo`) reports the training and testing scores of the outer CV, as well as the hyperparameters selected by the inner CV.

In [20]:
estimator = svm.SVR(kernel='precomputed')
estimator_param_grid = dict(C=[10**x for x in [-3, -2, -1, 0, 1, 2, 3]])
g = rbf_median_heruistic(X, clr=True, eps=1e-5)
train_scores, test_scores, selected_params = run_nested_cv_kfold(X, y, 
                                                                 estimator, estimator_param_grid, 
                                                                 scoring = 'neg_mean_squared_error', 
                                                                 kmat_fun=wrap(kmat_aitchison_rbf, g=g, c_X=1e-5, c_Y=1e-5), 
                                                                 center_kmat=False, 
                                                                 use_count=False, 
                                                                 eps=0,
                                                                 n_fold_outer=10, 
                                                                 n_fold_inner=5, 
                                                                 stratify=False, 
                                                                 shuffle=False, 
                                                                 random_state=seed_num, 
                                                                 n_jobs=4, 
                                                                 verbose=0)

#### Compare multiple kernels (and potentially RF or dummy baseline) with nested CV

In [42]:
# kernel models
g1 = rbf_median_heruistic(X, clr=False)
g2 = rbf_median_heruistic(X, clr=True, eps=1e-5)
kernel_params_dict = default_kernel_params_grid(g1=g1, g2=g2)
kmat_with_params = get_kmat_with_params(kernel_params_dict)

# add RF and baseline
mod_with_params = kmat_with_params
mod_with_params['RF use comp'] = None # omit if do not want to include
mod_with_params['baseline'] = None # omit if do not want to include

# hyperparameters
param_grid_svm = dict(C=[10**x for x in [-3, -2, -1, 0, 1, 2, 3]])
param_grid_rf = dict(n_estimators=[10, 20, 50, 100, 250, 500])
param_grid_baseline = dict(strategy=["mean", "median"])

# run comparison
train_scores_all, test_scores_all, selected_params_all = run_experiments(X,
                                                                         y,
                                                                         mod_with_params,
                                                                         param_grid_svm, 
                                                                         param_grid_rf, # set to None if do not want to include
                                                                         param_grid_baseline, # set to None if do not want to include
                                                                         center_kmat=False,
                                                                         outer_cv_type="kfold", 
                                                                         grp=None, 
                                                                         n_fold_outer=10,
                                                                         n_fold_inner=5,
                                                                         type='regression',
                                                                         scoring='neg_mean_squared_error',
                                                                         kernel_estimator='svm',
                                                                         n_jobs=-1,
                                                                         random_state=seed_num,
                                                                         verbose=0)

--- running: linear ---
average test score: -2.9191846205234326
--- running: rbf_g_2.4322502193296414 ---
average test score: -1.0946637450943297
--- running: rbf_g_2.957920564714544 ---
average test score: -1.1435724954217286
--- running: rbf_g_5.915841129429088 ---
average test score: -1.4811979282956333
--- running: rbf_g_14.388805884573213 ---
average test score: -1.8866491936864296
--- running: rbf_g_34.99717626864483 ---
average test score: -3.0794802018492766
--- running: rbf_g_85.12188965532951 ---
average test score: -5.378823711721106
--- running: rbf_g_59.158411294290886 ---
average test score: -4.312968349275321
--- running: rbf_g_591.5841129429089 ---
average test score: -7.104529409219178
--- running: generalized-js_a_1_b_0.5 ---
average test score: -0.6825715030917372
--- running: generalized-js_a_1_b_1 ---
average test score: -0.02607531057835826
--- running: generalized-js_a_10_b_0.5 ---
average test score: -0.1107408820797097
--- running: generalized-js_a_10_b_1 ---
a

#### Report results comparing multiple kernels (as well as RF and baseline)

In [43]:
# print full results
make_result_table(mod_with_params, train_scores_all, test_scores_all, selected_params_all)

avg_train_score  avg_test_score  \
kernel      kernel_params                                           
RF use comp NaN                         -0.196865       -1.432104   
aitchison   c_1e-06                     -0.547682       -0.620154   
            c_1e-07                     -0.547505       -0.620889   
            c_1e-05                     -0.548140       -0.621470   
            c_0.0001                    -0.551107       -0.626444   
...                                           ...             ...   
rbf         g_14.388805884573213        -0.048854       -1.886649   
            g_34.99717626864483         -0.009646       -3.079480   
            g_59.158411294290886        -0.009700       -4.312968   
            g_85.12188965532951         -0.009768       -5.378824   
            g_591.5841129429089         -0.009874       -7.104529   

                                 most_freq_best_param  
kernel      kernel_params                              
RF use comp NaN                                   500  
aitchison   c_1e-06                                 1  
            c_1e-07                                 1  
            c_1e-05                                 1  
            c_0.0001                                1  
...                                               ...  
rbf         g_14.388805884573213                   10  
            g_34.99717626864483                    10  
            g_59.158411294290886                   10  
            g_85.12188965532951                    10  
            g_591.5841129429089                    10  

[78 rows x 3 columns]

In [44]:
# print top models only
top_models_in_each_group(mod_with_params, train_scores_all, test_scores_all, selected_params_all, top_n=1, kernel_mod_only=False)

,estimator_key,kmat_fun,avg_test_score,most_freq_best_param
18,hilbertian_a_1_b_-1,<function wrap.<locals>.calc at 0x17ca70af0>,-0.023497,1000
10,generalized-js_a_1_b_1,<function wrap.<locals>.calc at 0x17ca70160>,-0.026075,1000
60,aitchison-rbf_c_0.0001_g_0.0008761356010002972,<function wrap.<locals>.calc at 0x17ca7d040>,-0.113358,1000
27,aitchison_c_1e-06,<function wrap.<locals>.calc at 0x17ca86820>,-0.620154,1
1,rbf_g_2.4322502193296414,<function wrap.<locals>.calc at 0x172a77dc0>,-1.094664,100
76,RF use comp,None,-1.432104,500
75,heat-diffusion_t_0.08753521870054244,<function wrap.<locals>.calc at 0x17ca7d8b0>,-1.985854,100
0,linear,<function kmat_linear at 0x15f0feaf0>,-2.919185,100
77,baseline,None,-7.142879,mean
